# Тема: Базовый SQL
# Проект: Исследование данных об инвестиции венчурных фондов

**Задача:**

Произвести различные выгрузки с помощью SQL.


**Исходные данные:**

База данных, которая хранит информацию о венчурных фондах и инвестициях в компании-стартапы (База основана на датасете Startup Investments опубликованном на платформе Kaggle).

## 1. Выясните количество закрывшихся компаний.

In [ ]:
query = '''SELECT COUNT(id)
           FROM company
           WHERE status = 'closed';
        '''

## 2. Выведите количество привлечённых средств для новостных компаний США. Необходимо использовать данные из таблицы `company`. Отсортируйте таблицу по убыванию значений в поле `funding_total` .

In [ ]:
query = '''SELECT funding_total
           FROM company
           WHERE country_code = 'USA'
                 AND category_code = 'news'
           ORDER BY funding_total DESC;
        '''

## 3. Посчитайте общую сумму сделок по покупке одних компаний другими в долларах. Отобразите только те сделки, которые осуществлялись за наличные с 2011 по 2013 год.

In [ ]:
query = '''SELECT SUM(price_amount)
           FROM acquisition
           WHERE term_code = 'cash'
                 AND EXTRACT(YEAR FROM CAST(acquired_at AS date)) IN (2011, 2012, 2013)
        '''

## 4. Выведите имя, фамилию и названия аккаунтов людей в твиттере. Названия  их аккаунтов должно начинаться на `Silver`.

In [ ]:
query = '''SELECT  first_name,
                   last_name,
                   twitter_username
           FROM people
           WHERE twitter_username LIKE 'Silver%'
        '''

## 5. Отобразите  всю информацию о людях, у которых названия аккаунтов в твиттере содержат подстроку `money`. Фамилия людей в выборке должна начинаться на `K`.

In [ ]:
query = '''SELECT id,  
                  first_name,
                  last_name,
                  company_id,
                  twitter_username,
                  created_at,
                  updated_at
           FROM people
           WHERE twitter_username LIKE '%money%'
                 AND last_name LIKE 'K%'
        '''

## 6. Отобразите общую сумму привлечённых инвестиций, которые получили компании, зарегистрированные в этой стране (для всех стран). Страну, в которой зарегистрирована компания, можно определить по коду страны.  Данные отсортируйте по убыванию суммы.

In [ ]:
query = '''SELECT country_code, 
                  SUM(funding_total)
           FROM company
           GROUP BY country_code
           ORDER BY SUM(funding_total) DESC;
        '''

## 7. Сформируйте таблицу с датой проведения раунда, а также минимальным и максимальным значением суммы инвестиций, привлечённых в эту дату.  В итоговой таблице оставьте только те записи, в которых минимальное значение суммы инвестиций не равно нулю и не равно максимальному значению.

In [ ]:
query = '''SELECT funded_at,
                  MIN(raised_amount),
                  MAX(raised_amount)
           FROM funding_round
           GROUP BY funded_at
           HAVING MIN(raised_amount) <> 0
                  AND MIN(raised_amount) <> MAX(raised_amount);
           '''

## 8. Создайте поле с тремя категориями:
* `high_activity` - фонды, которые инвестируют в 100 и более компаний;
* `middle_activity` - фонды, которые инвестируют в 20 и более компаний до 100;
* `low_activity` - количество инвестируемых компаний фонда не достигает 20.

Отобразите все поля таблицы `fund` и новое поле с категориями.

In [ ]:
query = '''SELECT *,
                  CASE
                      WHEN invested_companies >= 100 THEN 'high_activity'
                      WHEN invested_companies >= 20 AND invested_companies < 100 THEN 'middle_activity'
                      WHEN invested_companies < 20 THEN 'low_activity'
                  END
           FROM fund;
        '''

## 9. Для каждой из категорий, из предыдущего задания, посчитайте округлённое до ближайшего целого числа среднее количество инвестиционных раундов, в которых фонд принимал участие. Отобразите на экране категории и среднее число инвестиционных раундов. Отсортируйте таблицу по возрастанию среднего.

In [ ]:
query = '''SELECT CASE
                       WHEN invested_companies>=100 THEN 'high_activity'
                       WHEN invested_companies>=20 THEN 'middle_activity'
                       ELSE 'low_activity'
                  END AS activity,
                  ROUND(AVG(investment_rounds)) AS avg_rounds
           FROM fund
           GROUP BY activity
           ORDER BY avg_rounds ASC;
        '''

## 10. Создайте таблицу с десятью самыми активными инвестирующими странами. Активность страны определите по среднему количеству компаний, в которые инвестируют фонды этой страны.  Посчитайте минимальное, максимальное и среднее число компаний, в которые инвестировали фонды, основанные с 2010 по 2012 год включительно для каждой страны. Исключите из таблицы страны с фондами, у которых минимальное число компаний, получивших инвестиции, равно нулю. Отсортируйте таблицу по среднему количеству компаний от большего к меньшему, а затем по коду страны в лексикографическом порядке.

In [ ]:
query = '''SELECT country_code,
                  MIN(invested_companies),
                  MAX(invested_companies),
                  AVG(invested_companies)
           FROM fund
           WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) BETWEEN '2010' AND '2012'
           GROUP BY country_code
           HAVING MIN(invested_companies) <> 0
           ORDER BY AVG(invested_companies) DESC
           LIMIT 10;
        '''

## 11. Отобразите имена и фамилии всех сотрудников стартапов. Добавьте поле с названием учебного заведения, которое окончил сотрудник, если эта информация известна.

In [ ]:
query = '''SELECT first_name,
                  last_name,
                  instituition
           FROM people AS p
           LEFT OUTER JOIN education AS e ON p.id = e.person_id
        '''

## 12. Для всех компаний найдите количество учебных заведений, которые окончили её сотрудники. Выведите название компании и число уникальных названий учебных заведений. Составьте топ-5 по количеству университетов.

In [ ]:
query = '''SELECT c.name,
                  COUNT(DISTINCT e.instituition) AS inst
           FROM company AS c
           INNER JOIN people AS p ON c.id=p.company_id
           INNER JOIN education AS e ON p.id=e.person_id
           GROUP BY c.name
           ORDER BY inst DESC
           LIMIT 5;
        '''

## 13. Составьте список:
* закрытых компаний с уникальными названиями;
* их первый раунд финансирования оказался последним.

In [ ]:
query = '''SELECT  DISTINCT name
           FROM company AS c
           INNER JOIN funding_round AS fr ON c.id=fr.company_id
           WHERE status = 'closed'
             AND is_first_round = 1 
             AND is_last_round = 1;
        '''

## 14. На основе предыдущего задания составьте список уникальных номеров сотрудников, которые работают в отобранных компаниях.

In [ ]:
query = '''SELECT p.id
           FROM people AS p 
           WHERE company_id IN (SELECT id
                                FROM company 
                                WHERE id IN (SELECT company_id
                                             FROM funding_round
                                             WHERE is_first_round = 1 AND is_last_round = 1)
                                  AND status = 'closed');
        '''

## 15. Составьте таблицу, с уникальными номерами сотрудников из предыдущей задачи и учебным заведением, которое окончил сотрудник.

In [ ]:
query = '''SELECT  DISTINCT person_id,
                   instituition
           FROM education
           WHERE person_id IN (SELECT p.id
                               FROM people AS p 
                               WHERE company_id IN (SELECT id
                                                    FROM company 
                                                    WHERE id IN (SELECT company_id
                                                                 FROM funding_round
                                                                 WHERE is_first_round = 1 AND is_last_round = 1)
                                                      AND status = 'closed'))
           GROUP BY person_id, instituition;
        '''

## 16. Выясните количество учебных заведений для каждого сотрудника из предыдущего задания.

In [ ]:
query = '''SELECT  DISTINCT person_id,
                   COUNT(instituition)
           FROM education
           WHERE person_id IN (SELECT p.id
                               FROM people AS p 
                               WHERE company_id IN (SELECT id
                                                    FROM company 
                                                    WHERE id IN (SELECT company_id
                                                                 FROM funding_round
                                                                 WHERE is_first_round = 1 AND is_last_round = 1)
                                                    AND status = 'closed'))
           GROUP BY person_id;
        '''

## 17. Составьте таблицу, в которой будут содержаться следующие поля:
* название фонда;
* название компании;
* сумма инвестиций, которую привлекла компания в раунде.


В таблицу войдут данные о компаниях, в истории которых было больше шести важных этапов. Раунды финансирования должны проходилть в период с 2012 по 2013 год включительно.

In [ ]:
query = '''SELECT f.name AS name_of_fund,
                  c.name AS name_of_company,
                  fr.raised_amount AS amount
           FROM investment AS i
           LEFT OUTER JOIN company AS c ON c.id = i.company_id
           LEFT OUTER JOIN fund AS f ON f.id=i.fund_id
           INNER JOIN
           (SELECT *
            FROM funding_round
            WHERE EXTRACT(YEAR FROM CAST(funded_at AS date)) BETWEEN '2012' AND '2013') AS fr ON fr.id = i.funding_round_id
            WHERE c.milestones > 6;
        '''

## 18. Создайте таблицу с полями:
* название компании-покупателя;
* сумма сделки;
* название компании, которую купили;
* сумма инвестиций, вложенных в купленную компанию;
* доля, которая отображает, во сколько раз сумма покупки превысила сумму вложенных в компанию инвестиций, округлённая до ближайшего целого числа.
Не учитывайте те сделки, в которых сумма покупки равна нулю.
Отсортируйте таблицу по сумме сделки от большей к меньшей, а затем по названию купленной компании в лексикографическом порядке. Ограничьте таблицу первыми десятью записями.

In [ ]:
query = '''SELECT c.name,
                  a.price_amount,
                  c_second.name,
                  c_second.funding_total,
                  ROUND(a.price_amount/c_second.funding_total)
           FROM acquisition AS a
           LEFT OUTER JOIN company AS c ON a.acquiring_company_id = c.id
           LEFT OUTER JOIN company AS c_second ON a.acquired_company_id = c_second.id
           WHERE a.price_amount != 0
             AND c_second.funding_total != 0
           ORDER BY a.price_amount DESC,
                    c_second.name
           LIMIT 10;
        '''

## 19. Создайте таблицу, в которую войдут названия компаний из категории `social`, получившие финансирование с 2010 по 2013 год включительно. Выведите номер месяца, в котором проходил раунд финансирования.

In [ ]:
query = '''SELECT c.name,
                  EXTRACT(MONTH FROM CAST(funded_at AS date))
           FROM company AS c
           LEFT OUTER JOIN funding_round AS fr ON c.id=fr.company_id
           WHERE category_code = 'social'
             AND EXTRACT(YEAR FROM CAST(funded_at AS date)) BETWEEN '2010' AND '2013'
        '''

## 20. Отберите данные по месяцам с 2010 по 2013 год, когда проходили инвестиционные раунды. Сгруппируйте данные по номеру месяца. Получите таблицу с полями:
* номер месяца, в котором проходили раунды;
* количество уникальных названий фондов из США, которые инвестировали в этом месяце;
* количество компаний, купленных за этот месяц;
* общая сумма сделок по покупкам в этом месяце.

In [ ]:
query = '''WITH 
           ft_table AS (SELECT EXTRACT(MONTH FROM CAST(fr.funded_at AS date)) AS foundation_date,
                               COUNT(DISTINCT f.name) AS count_of_fund
                        FROM funding_round AS fr
                        LEFT JOIN investment AS i ON i.funding_round_id = fr.id
                        LEFT JOIN fund AS f ON i.fund_id = f.id
                        WHERE EXTRACT(YEAR FROM CAST(fr.funded_at AS date)) BETWEEN '2010' AND '2013'
                          AND f.country_code = 'USA'
                        GROUP BY foundation_date), 

           sd_table AS (SELECT EXTRACT(MONTH FROM CAST(acquired_at AS date)) AS deals_date,
                               COUNT(acquired_company_id) AS count_of_acquired,
                               SUM(price_amount) AS sum_of_acquired
                        FROM acquisition
                        WHERE EXTRACT(YEAR FROM CAST(acquired_at AS date)) BETWEEN '2010' AND '2013'
                        GROUP BY deals_date)
                   
                   
           SELECT  ft_table.foundation_date,
                   ft_table.count_of_fund,
                   sd_table.count_of_acquired,
                   sd_table.sum_of_acquired
           FROM ft_table
           JOIN sd_table ON ft_table.foundation_date = sd_table.deals_date;
        '''

## 21. Сделайте сводную таблицу и выведите среднюю сумму инвестиций для стран, в которых есть стартапы. Они должны быть зарегистрированны в 2011, 2012 и 2013 годах. Данные за каждый год должны отобразите в отдельном поле. Отсортируйте таблицу по среднему значению инвестиций за 2011 год от большего к меньшему.

In [ ]:
query = '''WITH
           as_11 AS (SELECT country_code,
                            AVG(funding_total) AS avg_sum_2011
                     FROM company
                     WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) = '2011'
                     GROUP BY country_code),

           as_12 AS (SELECT country_code,
                            AVG(funding_total) AS avg_sum_2012
                     FROM company
                     WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) = '2012'
                     GROUP BY country_code),

           as_13 AS (SELECT country_code,
                            AVG(funding_total) AS avg_sum_2013
                     FROM company
                     WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) = '2013'
                     GROUP BY country_code)


           SELECT  as_11.country_code,
                   as_11.avg_sum_2011,
                   as_12.avg_sum_2012,
                   as_13.avg_sum_2013
           FROM as_11 INNER JOIN as_12 ON as_11.country_code=as_12.country_code
           INNER JOIN as_13 ON as_12.country_code=as_13.country_code
           ORDER BY avg_sum_2011 DESC;
        '''